In [3]:
# Datetime
from datetime import datetime
import time

import matplotlib.pyplot as plt
from pandas import DataFrame
import pandas as pd

import warnings

import pennylane as qml
import numpy as np

# Saving
import pickle
import os
import copy

# sklearn
from sklearn.preprocessing import StandardScaler

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

def generate_narma_data(n_samples, order, seed=None):
    """
    Generates NARMA time-series data.
    """
    # Fix the random seed for reproducibility
    if seed is not None:
        np.random.seed(seed)

    u = np.random.uniform(0, 0.5, n_samples)
    y = np.zeros(n_samples)

    for t in range(order, n_samples):
        term1 = 0.3 * y[t-1]
        term2 = 0.05 * y[t-1] * np.sum(y[t-i-1] for i in range(order))
        term3 = 1.5 * u[t-order] * u[t-1]
        term4 = 0.1
        y[t] = term1 + term2 + term3 + term4
        
    return y.reshape(-1, 1)

def transform_narma_data(data, seq_length):
    """
    Transforms NARMA data into input-output pairs for sequence prediction.
    """
    x = []
    y = []

    for i in range(len(data) - seq_length - 1):
        _x = data[i:(i + seq_length)]
        _y = data[i + seq_length]
        x.append(_x)
        y.append(_y)

    x = torch.from_numpy(np.array(x)).float()
    y = torch.from_numpy(np.array(y)).float()
    
    return x, y

def get_narma_data(n_samples=240, order=10, seq_length=10, seed=None):
    """
    Generates and transforms NARMA data for the QLSTM model.
    """
    seed = seed
    print(f"seed: {seed}")

    # Generate NARMA data
    narma_series = generate_narma_data(n_samples, order, seed=seed)

    # Normalize the dataset
    scaler = MinMaxScaler(feature_range=(-1, 1))
    dataset = scaler.fit_transform(narma_series)

    # Transform data into sequences
    x, y = transform_narma_data(dataset, seq_length)
    
    return x, y 

In [5]:
x_data, y_data = get_narma_data()
print("Shape of X data:", x_data.shape)
print("Shape of Y data:", y_data.shape)

seed: None
Shape of X data: torch.Size([229, 10, 1])
Shape of Y data: torch.Size([229, 1])


/tmp/ipykernel_995575/2578747515.py:18: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  term2 = 0.05 * y[t-1] * np.sum(y[t-i-1] for i in range(order))


In [ ]:
torch.manual_seed(0)

dtype = torch.DoubleTensor

# x, y = get_narma_data(seq_length=10)
print("Getting NARMA data...")
x, y = get_narma_data(n_samples=240, seq_length=10, seed=2025)

# num_for_train_set = int(0.67 * len(x))

# x_train = x[:num_for_train_set].type(dtype)
# y_train = y[:num_for_train_set].type(dtype)

# x_test = x[num_for_train_set:].type(dtype)
# y_test = y[num_for_train_set:].type(dtype)

# Split data into training, validation, and testing sets (70%, 15%, 15%)
train_end_idx = int(0.70 * len(x))
val_end_idx = int(0.85 * len(x)) # 70% + 15%

x_train = x[:train_end_idx].type(dtype)
y_train = y[:train_end_idx].type(dtype)

x_val = x[train_end_idx:val_end_idx].type(dtype)
y_val = y[train_end_idx:val_end_idx].type(dtype)

x_test = x[val_end_idx:].type(dtype)
y_test = y[val_end_idx:].type(dtype)

print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("x_test.shape: ", x_test.shape)
print("y.shape: {}".format(y.shape))

In [ ]:

# Check the trainable parameters
print("Show the parameters in QLSTM.")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter name: {name}")
        print(f"Parameter shape: {param.shape}")
        # print(f"Parameter grad: {param.grad}")
        # print(f"Parameter value: {param.data}\n")

##

exp_name = "QLSTM_TS_MODEL_NARMA_1_order_5_batch_32"
exp_index = 1
train_len = len(x_train)


opt = torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

train_loss_for_all_epoch = []
val_loss_for_all_epoch = [] 
test_loss_for_all_epoch = []
iteration_list = []

for i in range(50):
    start_epoch_time = time.time() # Start timer

    iteration_list.append(i + 1)
    train_loss_epoch = train_epoch_full(opt = opt, model = model, X = x_train, Y = y_train, batch_size = 32)

    # Calculate validation loss
    val_loss = nn.MSELoss()
    model_res_val, _ = model(x_val)
    val_loss_val = val_loss(model_res_val.transpose(0,1)[-1], y_val).detach().numpy()

    # Calculate test loss
    test_loss = nn.MSELoss()
    model_res_test, _ = model(x_test)
    test_loss_val = test_loss(model_res_test.transpose(0,1)[-1], y_test).detach().numpy() # 2024 11 11: .transpose(0,1)

    end_epoch_time = time.time() # End timer
    epoch_duration = end_epoch_time - start_epoch_time

    print("Epoch {} finished in {:.2f} seconds".format(i, epoch_duration))
    print("TRAIN LOSS at {}-th epoch: {}".format(i, train_loss_epoch))
    print("VAL LOSS at {}-th epoch: {}".format(i, val_loss_val))
    print("TEST LOSS at {}-th epoch: {}".format(i, test_loss_val))

    train_loss_for_all_epoch.append(train_loss_epoch)
    val_loss_for_all_epoch.append(val_loss_val)
    test_loss_for_all_epoch.append(test_loss_val)

    # Run the test
    test_run_res, _ = model(x.type(dtype))
    total_res = test_run_res.transpose(0,1)[-1].detach().cpu().numpy() # 2024 11 11: .transpose(0,1)
    ground_truth_y = y.clone().detach().cpu()

    saving(
            exp_name = exp_name, 
            exp_index = exp_index, 
            train_len = train_len, 
            val_end_idx = val_end_idx,  
            iteration_list = iteration_list, 
            train_loss_list = train_loss_for_all_epoch, 
            val_loss_list = val_loss_for_all_epoch,
            test_loss_list = test_loss_for_all_epoch, 
            model = model, 
            simulation_result = total_res, 
            ground_truth = ground_truth_y)
    
# --- CSV LOGGING ---
# Prepare data for CSV files
file_name_prefix = exp_name + "_NO_" + str(exp_index) + "_Epoch_" + str(iteration_list[-1])

epoch_log_data = {
    'epoch': iteration_list,
    'train_loss': train_loss_for_all_epoch,
    'validation_loss': val_loss_for_all_epoch,
    'test_loss': test_loss_for_all_epoch
}

timeseries_log_data = {
    'prediction': total_res.flatten(),
    'ground_truth': ground_truth_y.numpy().flatten()
}

save_log_to_csv(exp_name, file_name_prefix, epoch_log_data, timeseries_log_data)

return